In [ ]:
!pip install -U spacy -q
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [ ]:

import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [ ]:
import json

# Define the filename
filename = 'training_data.json'
filename = '/content/training_data_3490.json'

# Initialize TRAIN_DATA as an empty list
TRAIN_DATA = []

# Open the file using 'with' statement to ensure it is properly closed
with open(filename, 'r') as f:
    # Load the JSON data into TRAIN_DATA
    TRAIN_DATA = json.load(f)

print(len(TRAIN_DATA['annotations']))

3490


In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 3490/3490 [00:00<00:00, 3969.97it/s]


create config.cfg file and save in directory

In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     61.00    7.75    6.06   10.74    0.08
  0     200         71.17   2055.41   99.87   99.89   99.86    1.00
  0     400          5.63      5.06   99.97   99.97   99.97    1.00
  1     600          4.28      5.93   99.97   99.97   99.97    1.00
  2     800         20.49     14.12   99.97   99.97   99.97    1.00
  3    1000          0.50      2.00   99.97   99.97   99.97    1.00
  4    1200          4.04      4.58   99.97   99.97   99.97    1.00
  6    1400         26.82     22.08   99.96   99.99   99.93    1.00
  7    1600          7.73      7.06 

In [ ]:
model_path = "/content/model-last"
nlp_ner = spacy.load(model_path)


In [ ]:
# input sample text
test_data = """
MCH	27.2	pg	27-32	Calculated
VLDL CHOLESTEROL	18.4	mg/dL	<30	Calculated
PLATELET COUNT	270000	cells/cu.mm	150000-410000	Electrical impedence
ERYTHROCYTE SEDIMENTATION	34	mm at the end	0-20	Modified Westegren
HBA1C, GLYCATED HEMOGLOBIN ,	6.3	%	HPLC
HDL CHOLESTEROL	48	mg/dL	40-60	Enzymatic
TOTAL CHOLESTEROL	238	mg/dL	<200	CHO-POD
LDL CHOLESTEROL	171.4	mg/dL	<100	Calculated
TRIGLYCERIDES	92	mg/dL	<150	GPO-POD

Triglycerides	92	mg/dL	<150	GPO-POD
NON-HDL CHOLESTEROL	190	mg/dL	<130	Calculated
TRI-IODOTHYRONINE (T3, TOTAL)	1.08	ng/mL	0.64-1.52	CMIA
THYROXINE (T4, TOTAL)	9.16	µg/dL	4.87-11.72	CMIA
THYROID STIMULATING HORMONE	1.710	µIU/mL	0.35-4.94	CMIA
BLOOD UREA NITROGEN	8.4	mg/dL	8.0 - 23.0	Calculated
CREATININE	0.64	mg/dL	0.72 – 1.18	JAFFE METHOD
Creatinine	0.64	mg/dL	0.72 – 1.18	JAFFE METHOD
UREA	18.00	mg/dL	17-43	GLDH, Kinetic Assay
URIC ACID	4.38	mg/dL	2.6-6.0	Uricase PAP
CALCIUM	9.20	mg/dL	8.8-10.6	Arsenazo III
VITAMIN B12 , SERUM	147	pg/mL	120-914	CLIA
PHOSPHORUS, INORGANIC	3.23	mg/dL	2.5-4.5	Phosphomolybdate
VITAMIN D (25 - OH VITAMIN D) , SERUM	10.1	ng/mL	CMIA
SODIUM	137	mmol/L	136–146	ISE (Indirect)
"""

In [ ]:
test_data = "Triglycerides	92	mg/dL	<150	GPO-POD mmol/L Phosphomolybdate"
doc = nlp_ner(test_data, )
dict_ = {'TEST_NAME': [],
         "UNIT":[]}
for entity in doc.ents:
    print(entity.text, entity.label_)
    if entity.label_=='TEST NAME':
        d1 = dict_['TEST_NAME']
        d1.append(entity.text)
        dict_['TEST_NAME'] = d1
    if entity.label_=='UNIT':
        d2 = dict_['UNIT']
        d2.append(entity.text)
        dict_['UNIT'] = d2
print(dict_)

Triglycerides TEST NAME
mg/dL UNIT
POD mmol/L Phosphomolybdate TEST NAME


{'TEST_NAME': ['Triglycerides', 'POD mmol/L Phosphomolybdate'],
 'UNIT': ['mg/dL']}

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
doc = nlp_ner(test_data) # input sample text
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
text_file_path = r'/content/All_data_1.txt'

with open(text_file_path, 'r') as fp:
    data = fp.readlines()

print("Number of lines: ", len(data))

In [ ]:
len(data)

3834

In [ ]:
for idx_, text_line in enumerate(data[:10]):
    print(idx_, text_line)
    doc = nlp_ner(text_line, )
    spacy.displacy.render(doc, style="ent", jupyter=True)
    dict_ = {'TEST_NAME': [],
         "UNIT":[]}
    for entity in doc.ents:
        # print(entity.text, entity.label_)
        if entity.label_=='TEST NAME':
            d1 = dict_['TEST_NAME']
            d1.append(entity.text)
            dict_['TEST_NAME'] = d1
        if entity.label_=='UNIT':
            d2 = dict_['UNIT']
            d2.append(entity.text)
            dict_['UNIT'] = d2
    print(dict_)
    print("______"*10)
    print()

0 RBC COUNT	5.12	Million/cu.mm	3.8-4.8	Electrical Impedence



{'TEST_NAME': ['RBC COUNT'], 'UNIT': ['Million/cu.mm']}
____________________________________________________________

1 HAEMOGLOBIN	13.9	g/dL	12-15	Spectrophotometer



{'TEST_NAME': ['HAEMOGLOBIN'], 'UNIT': ['g/dL']}
____________________________________________________________

2 TOTAL IRON BINDING CAPACITY ( TIBC)	325	µg/dL	261-462	TPTZ AND NITROSO-



{'TEST_NAME': ['TOTAL IRON BINDING CAPACITY ( TIBC)'], 'UNIT': ['µg/dL']}
____________________________________________________________

3 IRON	89.0	µg/dL	60-180	TPTZ



{'TEST_NAME': ['IRON'], 'UNIT': ['µg/dL']}
____________________________________________________________

4 GAMMA GLUTAMYL TRANSPEPTIDASE	26.00	U/L	<38	IFCC



...

{'TEST_NAME': ['BILIRUBIN CONJUGATED (DIRECT)'], 'UNIT': ['mg/dL']}
____________________________________________________________

6 PCV	40.50	%	36-46	Electronic pulse &



{'TEST_NAME': ['PCV'], 'UNIT': ['%']}
____________________________________________________________

7 MCV	79.1	fL	83-101	Calculated



{'TEST_NAME': ['MCV'], 'UNIT': ['fL']}
____________________________________________________________

8 MCHC	34.4	g/dL	31.5-34.5	Calculated



{'TEST_NAME': ['MCHC'], 'UNIT': ['g/dL']}
____________________________________________________________

9 R.D.W	12.9	%	11.6-14	Calculated



{'TEST_NAME': ['R.D.W'], 'UNIT': ['%']}
____________________________________________________________



Try first

In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     57.40    2.89    1.99    5.27    0.03
  0     200         54.61   1972.18   99.99   99.98  100.00    1.00
  1     400          0.75      1.05  100.00  100.00  100.00    1.00
  2     600          0.10      0.13  100.00  100.00  100.00    1.00
  3     800          0.00      0.00  100.00  100.00  100.00    1.00
  5    1000          0.00      0.00  100.00  100.00  100.00    1.00
  7    1200          0.66      0.32  100.00  100.00  100.00    1.00
  9    1400          0.00      0.00  100.00  100.00  100.00    1.00
 12    1600          0.00      0.00 

In [ ]:
model_path = "/content/model-last"
nlp_ner = spacy.load(model_path)

In [ ]:

# import spacy
# import sys
# from collections import defaultdict

# nlp = spacy.load('en')
# text = u'Will Japan join the European Union? If yes, we should \
# move to United States. Fasten your belts, America we are coming'


# with nlp.disable_pipes('ner'):
#     doc = nlp(text)

# test_data = """
# Triglycerides	92	mg/dL	<150	GPO-POD
# """
# # doc = nlp_ner(test_data) # input sample text

# nlp1 = spacy.load('en_core_web_sm')
# doc = nlp_ner(test_data) # input sample text
# threshold = 0.2
# (beams, somethingelse) = nlp.entity.beam_parse([ doc ], beam_width = 16, beam_density = 0.0001)

# entity_scores = defaultdict(float)
# for beam in beams:
#     for score, ents in nlp.entity.moves.get_beam_parses(beam):
#         for start, end, label in ents:
#             entity_scores[(start, end, label)] += score

# print ('Entities and scores (detected with beam search)')
# for key in entity_scores:
#     start, end, label = key
#     score = entity_scores[key]
#     if ( score > threshold):
#         print ('Label: {}, Text: {}, Score: {}'.format(label, doc[start:end], score))

In [ ]:
test_data = '''MCH	27.2	pg	27-32	Calculated
VLDL CHOLESTEROL	18.4	mg/dL	<30	Calculated
PLATELET COUNT	270000	cells/cu.mm	150000-410000	Electrical impedence
ERYTHROCYTE SEDIMENTATION	34	mm at the end	0-20	Modified Westegren
HBA1C, GLYCATED HEMOGLOBIN ,	6.3	%	HPLC
HDL CHOLESTEROL	48	mg/dL	40-60	Enzymatic
TOTAL CHOLESTEROL	238	mg/dL	<200	CHO-POD
LDL CHOLESTEROL	171.4	mg/dL	<100	Calculated
TRIGLYCERIDES	92	mg/dL	<150	GPO-POD
NON-HDL CHOLESTEROL	190	mg/dL	<130	Calculated
TRI-IODOTHYRONINE (T3, TOTAL)	1.08	ng/mL	0.64-1.52	CMIA
THYROXINE (T4, TOTAL)	9.16	µg/dL	4.87-11.72	CMIA
THYROID STIMULATING HORMONE	1.710	µIU/mL	0.35-4.94	CMIA
BLOOD UREA NITROGEN	8.4	mg/dL	8.0 - 23.0	Calculated
CREATININE	0.64	mg/dL	0.72 – 1.18	JAFFE METHOD
UREA	18.00	mg/dL	17-43	GLDH, Kinetic Assay
URIC ACID	4.38	mg/dL	2.6-6.0	Uricase PAP
CALCIUM	9.20	mg/dL	8.8-10.6	Arsenazo III
VITAMIN B12 , SERUM	147	pg/mL	120-914	CLIA
PHOSPHORUS, INORGANIC	3.23	mg/dL	2.5-4.5	Phosphomolybdate
VITAMIN D (25 - OH VITAMIN D) , SERUM	10.1	ng/mL	CMIA
SODIUM	137	mmol/L	136–146	ISE (Indirect)
'''

In [ ]:
doc = nlp_ner(test_data) # input sample text

In [ ]:
# display in Jupyter
test_data = """
Triglycerides	92	mg/dL	<150	GPO-POD
"""
doc = nlp_ner(test_data) # input sample text
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
doc.user_data

{}

In [ ]:
# Iterate over the entities in the Doc object
for entity in doc.ents:
    print(entity.text, entity.label_)

Triglycerides TEST NAME
mg/dL UNIT


In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Triglycerides TEST NAME
mg/dL UNIT


In [ ]:
# Print the entities along with their labels and probabilities
for ent in doc.ents:
    print(ent.text, ent.label_, ent.label, ent.start_char, ent.end_char, ent.start, ent.end, )

Triglycerides TEST NAME 13542168876943382919 1 14 1 2
mg/dL UNIT 10132720570187401599 18 23 5 8


In [ ]:
# display in Jupyter

spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
# display in Jupyter

spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:



import pandas as pd


In [ ]:
# df = pd.read_excel('/content/AllKeywords_030424.xlsx')
pt = '/content/AllKeywords_050424.pkl'
df = pd.read_pickle(pt)

In [ ]:
df.columns

Index(['GroupName', 'TestID', 'TestName', 'UnitID', 'UnitName',
       'AltTestNamesStr', 'AltTestIDcount', 'AltUnitNamesStr',
       'AltUnitIDcount', 'RangeMale', 'RangeFemale', 'AltTestID', 'AltUnitID',
       'AltTestNamesList', 'AltTestIDList', 'AltUnitNamesList',
       'AltUnitIDList'],
      dtype='object')

In [ ]:
idx_ = df[df['TestID']==56].index[0]

55

In [ ]:
idx_ = df[df['TestID']==56].index[0]
d2 = df.loc[55,['TestName', 'AltTestNamesList', 'UnitName', 'AltUnitNamesList']]

In [ ]:
import json

# Define the filename
filename = 'training_data.json'

# Initialize TRAIN_DATA as an empty list
TRAIN_DATA = []

# Open the file using 'with' statement to ensure it is properly closed
with open(filename, 'r') as f:
    # Load the JSON data into TRAIN_DATA
    TRAIN_DATA = json.load(f)

print(len(TRAIN_DATA['annotations']))

156


In [ ]:
pt = '/content/AllKeywords_050424.pkl'
df = pd.read_pickle(pt)

def get_data(TestID):
    idx_ = df[df['TestID']==TestID].index[0]
    data = {}
    data['TestID'] = TestID
    d2 = df[['TestName', 'AltTestNamesList', 'UnitName', 'AltUnitNamesList']]
    for i in range(len(d2)):
        TestName_list = [d2.iloc[i, 0]]
        AltTestNamesList = d2.iloc[i, 1]
        TestName_list.extend(AltTestNamesList)
        d2.iloc[i, 1] =  TestName_list
        UnitName_list = [d2.iloc[i, 2]]
        AltUnitNamesList = d2.iloc[i, 3]
        UnitName_list.extend(AltUnitNamesList)
        d2.iloc[i, 3] =  UnitName_list

    AltTestNamesList  = list(d2.iloc[idx_, 1])
    # print(len(AltTestNamesList), AltTestNamesList )
    AltUnitNamesList  = list(d2.iloc[idx_, 3])
    # print(len(AltUnitNamesList), AltUnitNamesList)
    # print("Total outcomes: ", (len(AltTestNamesList)*len(AltUnitNamesList)))
    data['AltTestNamesList'] = AltTestNamesList
    data['AltUnitNamesList'] = AltUnitNamesList
    data['TotalOutcomes'] = (len(AltTestNamesList)*len(AltUnitNamesList))
    return data

In [ ]:

test_id = 25
data = get_data(test_id )
data


<ipython-input-70-821dbb32d4df>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2.iloc[i, 1] =  TestName_list
<ipython-input-70-821dbb32d4df>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2.iloc[i, 3] =  UnitName_list


{'TestID': 25,
 'AltTestNamesList': ['Triglycerides',
  'TG',
  'TRIGLYCERIDES, Serum',
  'S. Triglycerides',
  'Serum Triglyceride',
  'Triglyceride',
  'TRIGLYCERIDES, SERUM',
  'Triglycerides (TGL) - Serum',
  'Triglycerides-Serum',
  'SR. TRIGLYCERIDES'],
 'AltUnitNamesList': ['mg/dl',
  'mg/dL',
  'mg%',
  'mg/dl',
  'mg/d L',
  'mg / dl',
  'mg/L'],
 'TotalOutcomes': 70}

In [ ]:


# 1: BMI sentance not found

# 2: BP	120/82 MMHG	mm/Hg
# sent = f"{test_nm}	120/82 MMHG	{unit}"

# 3:    RBC COUNT	5.21	Million/cu.mm	4.5-5.5	Electrical Impedence
# sent = f"{test_nm}	5.21	{unit}	4.5-5.5	Electrical Impedence"

# 4: HEMOGLOBIN (HB)	13.8	13.0 - 17.0	g/dL
# sent = f"{test_nm}	13.8	13.0 - 17.0	{unit}"

# 5: HEMATOCRIT (PCV)	43.6	40 - 50	%
# sent = f"{test_nm}	43.6	40 - 50	{unit}"

# 6: MEAN CORPUSCULAR VOLUME (MCV)	92.0	83 - 101	fL
# sent = f"{test_nm}	92.0	83 - 101	{unit}"

# 7: MCHC (Mean Corpuscular Hemoglobin	30.5	g/dl	31.5-34.5
# sent = f"{test_nm}	30.5	{unit}	31.5-34.5"

# 8: RED CELL DISTRIBUTION WIDTH (RDW)	13.7	11.6 - 14.0	%
# sent = f"{test_nm}	13.7	11.6 - 14.0	{unit}"

# 9: RDW-SD	40.70	FL	39.0-46.0
# sent = f"{test_nm}	40.70	{unit}	39.0-46.0"

# 10: WHITE BLOOD CELL COUNT	6.38	4.0 - 10.0	thou/µL
# sent = f"{test_nm}	6.38	4.0 - 10.0	{unit}"

# 11: NEUTROPHILS	59	40 - 80	%
# sent = f"{test_nm}	59	40 - 80	{unit}"

# 12: LYMPHOCYTES	33	20 - 40	%
# sent = f"{test_nm}	33	20 - 40	{unit}"

# 13: MONOCYTES	7	2.0 - 10.0	%
# sent = f"{test_nm}	7	2.0 - 10.0	{unit}"

# 14: EOSINOPHILS	1	1.0 - 6.0	%
# sent = f"{test_nm}	1	1.0 - 6.0	{unit}"

# 15: BASOPHILS	0	0 - 2	%
# sent = f"{test_nm}	0	0 - 2	{unit}"

# 16: PLATELET COUNT	223000	cells/cu.mm	150000-410000	Electrical impedence
# sent = f"{test_nm}	223000	{unit}	150000-410000	Electrical impedence"

# E.S.R (WINTROBE)	26	mm/1st	0-9 ?
# 17: ERYTHROCYTE SEDIMENTATION	3	mm at the end	0-15	Modified Westergren
# sent = f"{test_nm}	3	{unit}	0-15	Modified Westergren"

# 18: BLOOD SUGAR FASTING	98.0	mg/dl	70-110
# sent = f"{test_nm}	98.0	{unit}	70-110"

# 19: GLUCOSE, POST PRANDIAL (PP), 2	103	mg/dL	70-140	HEXOKINASE
# sent = f"{test_nm} 2	103	{unit}	70-140	HEXOKINASE"

# 20: 'Random Blood Sugar'
# sentance not found
# sent = f"{test_nm} 2	103	{unit}	70-140"

# 21: 'GLYCOSYLATED HEMOGLOBIN (HBA1C)	5.6	Non-diabetic: < 5.7	%'
# sent = f"{test_nm}	5.6	Non-diabetic: < 5.7	{unit}"

# 22: CHOLESTEROL	141	< 200 Desirable	mg/dL
# sent = f"{test_nm}	141	< 200 Desirable	{unit}"

# 23: HDL CHOLESTEROL	36	Low	Low HDL Cholesterol <40	mg/dL
# sent = f"{test_nm}	36	Low	Low HDL Cholesterol <40	{unit}"

# 24: DIRECT LDL CHOLESTEROL	93.00	Optimal : < 100	mg/dL
# sent = f"{test_nm}	93.00	Optimal : < 100	{unit}"

# 25: TRIGLYCERIDES	245	mg/dL	<150	Enzymatic
# sent = f"{test_nm}	245	{unit}	<150	Enzymatic"






# 38:   VITAMIN D (25 - OH VITAMIN D) , SERUM	7.93	ng/mL	CLIA
# sent = f"{test_nm}	28.2	{unit}	30 -100	CLIA"


# 2013

In [ ]:
AltTestNamesList = data['AltTestNamesList']
AltUnitNamesList = data['AltUnitNamesList']
TotalOutcomes = data['TotalOutcomes']

all_sent_for_test = []
for test_nm in AltTestNamesList:
    for unit in AltUnitNamesList:
        sent = f"{test_nm}	120/82 MMHG	{unit}"
        sent = f"{test_nm}	245	{unit}	<150	Enzymatic"
        len_test = len(test_nm)
        unit_start = len_test+len('	245	')
        unit_end = unit_start+len(unit)
        sent_ = [sent, {'entities': [[0, len_test, 'TEST NAME'], [unit_start, unit_end, 'UNIT']]}]
        all_sent_for_test.append(sent_)

if TotalOutcomes==len(all_sent_for_test):
    print(len(all_sent_for_test))
    print("All test sentence created......")
else:
    print("Some test sentance missing")
    print("TotalOutcomes: ", TotalOutcomes)
    print("length of list 'all_sent_for_test': ", len(all_sent_for_test))

70
All test sentence created......


In [ ]:
# Define the filename
filename = 'training_data.json'

# Initialize TRAIN_DATA as an empty list
TRAIN_DATA = []
# Open the file using 'with' statement to ensure it is properly closed
with open(filename, 'r') as f:
    # Load the JSON data into TRAIN_DATA
    TRAIN_DATA = json.load(f)

# Before append length of ann list
bf = len(TRAIN_DATA['annotations'])
print(len(TRAIN_DATA['annotations']))
# append to previous annotations

TRAIN_DATA['annotations'].extend(all_sent_for_test)
# After append length of ann list
af = len(TRAIN_DATA['annotations'])
print(len(TRAIN_DATA['annotations']))
print((af- bf))

# Save the updated data back to the same dictionary
with open(filename, 'w') as f:
    json.dump(TRAIN_DATA, f)

1943
2013
70


In [ ]:
AltTestNamesList = data['AltTestNamesList']
AltUnitNamesList = data['AltUnitNamesList']
ml = []
for i in AltTestNamesList:
    for j in AltUnitNamesList:
        sent = f"{i}	7.70	{j}	6.6-8.3	Biuret"
        sent_ = [sent, {'entities': [[0, len(i), 'TEST NAME'], [len(i)+6, len(i)+6+len(j), 'UNIT']]}]
        ml.append(sent_)

In [ ]:
'VITAMIN D (25 - OH VITAMIN D) , SERUM	28.2	ng/mL	30 -100	CLIA'

78

In [ ]:
ml[0]

['Protein Total\t7.70\tg/dl\t6.6-8.3\tBiuret',
 {'entities': [[0, 13, 'TEST NAME'], [19, 23, 'UNIT']]}]

In [ ]:
'Protein Total\t7.70\tg/dl\t6.6-8.3\tBiuret'[19:23]

'g/dl'

In [ ]:
TRAIN_DATA['annotations'][0]

['RBC COUNT\t5.12\tMillion/cu.mm\t3.8-4.8\tElectrical Impedence\r',
 {'entities': [[0, 9, 'TEST NAME'], [15, 28, 'UNIT']]}]